# Practical 9: Applying the backward Euler method in the Black-Scholes

model

Computational Finance with Python

[Alet Roux](https://www.york.ac.uk/maths/staff/alet-roux/) ([Department
of Mathematics](https://maths.york.ac.uk), University of York)

Click on the following to open this file in Google Colab:

<figure>
<a
href="https://colab.research.google.com/github/aletroux/comp-finance-python/blob/main/practicals/P09_backward_Euler_Black-Scholes_equation_prac.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg"
alt="Open In Colab" /></a>
<figcaption>Open In Colab</figcaption>
</figure>

The aim of this practical is to explore the backward Euler method in the
context of the Black-Scholes partial differential equation
$$\frac{\partial V}{\partial \tau}(\tau,S) = rS\frac{\partial V}{\partial S}(\tau,S) + \frac{1}{2}\sigma^2 S^2\frac{\partial^2 V}{\partial S^2}(\tau,S)  -r V(\tau,S)$$
for $t\in[0,T]$ and $S>0$, coupled with suitable initial and boundary
conditions. The focus in this practical is on implementing different
types of boundary conditions.

# 1. Butterfly option (homogeneous Dirichlet boundary condition)

The payoff of a *butterfly option* with strike $K$ and spread $L$ is as
follows: $$ f(S) =
\begin{cases}
S - K + L & \text{if } S \in [K-L,K], \\
K + L - S & \text{if } S \in (K, K+L], \\
0 & \text{otherwise}.
\end{cases}$$ The following Python function implements the payoff of a
butterfly option. A graphical representation is also provided for a
butterfly option with $K=100$ and $L=20$.

In [1]:
import numpy as np

def butterfly_payoff(S, K, L):
    """calculates payoff of butterfly option with strike K and spread L when stock price is S"""
    if K - L < S <= K:
        return S - K + L
    elif K < S < K + L:
        return K + L - S
    else:
        return 0

K = 100
L = 20
S = np.linspace(50,150,500)

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (12,7))

ax.plot(S, np.vectorize(butterfly_payoff, otypes=[np.float64])(S, K, L))

# plot cosmetics
ax.set(title = f"Payoff of butterfly option with K={K} and L={L}")
ax.set(xlabel='$S$', xlim=(50, 150))
ax.xaxis.grid(True)
ax.yaxis.grid(True)

Consider a butterfly option that can be exercised at time $T=1$. The
form of the payoff suggests that the value function $V$ satisfies
$$ \lim_{S\rightarrow 0} V(\tau, S) = 0 = \lim_{S\rightarrow \infty} V(\tau, S).$$

The following code implements the backward Euler method for the
Black-Scholes partial differential equation in the Black-Scholes model
with $S_0=90$, $T=0.5$, $r=0.03$, $\sigma=0.15$. We will use these
parameters throughout the practical.

Study it carefully to see how it works.

In [2]:
import numpy as np
import math
import scipy.sparse as ssparse

def FD_BE_price (Smin, Smax, T, r, sigma, payoff, M, N):    
    """Backward Euler finite difference approximation of option price in Black-Scholes model
    with zero at the boundary.
    Arguments:
        Smin: lower boundary of grid
        Smax: upper boundary of grid
        T: maturity date
        r: interest rate
        sigma: constant volatility
        payoff: payoff function
        M: number of steps in direction tau
        N: number of steps in direction S
    Returns:
        V: matrix of results, V[i,j] is value at 
            iT/M and Smin + j(Smax-Smin)/N
        tau: vector of time steps
        S: vector of stock prices
    """

    V = np.zeros((M+1,N+1))

    # discretization parameters
    tau = np.linspace(0, T, M+1)
    S = np.linspace(Smin, Smax, N+1)
    Sint = S[1:-1]    
    dtau = tau[1]-tau[0]
    dS = S[1] - S[0]
    
    #initial value
    V[0] = np.vectorize(payoff, otypes=[np.float64])(S)

    # parameters that don't depend on time step
    alpha = dtau/2/dS**2 * sigma**2 * Sint**2
    beta = r*dtau/2/dS*Sint

    # Backward Euler matrix
    u = - alpha - beta
    d = 2*alpha + r*dtau + 1
    l = beta - alpha
    A_BE = ssparse.diags(diagonals=[u[:-1], d, l[1:]], offsets=[1, 0, -1], shape=(N-1, N-1), 
                         format='csc')
    # the above is equivalent to the following two lines:
    # A_BE = ssparse.diags(diagonals=[u[:-1], d, l[1:]], offsets=[1, 0, -1], shape=(N-1, N-1))
    # A_BE = A_BE.tocsc()

    # create solver function
    A_BE_solver = ssparse.linalg.factorized(A_BE)

    for i in range(M):
        V[i+1, 1:-1] = A_BE_solver(V[i, 1:-1])

    return V, tau, S
                    
M = 100
N = 100

Smin = 50
Smax = 150

T = 0.5
r = 0.03
sigma = 0.15

V, tau, S = FD_BE_price (Smin, Smax, T, r, sigma, lambda S: butterfly_payoff(S, K, L), M, N)


In [3]:
Ss, taus = np.meshgrid(S, tau)

fig = plt.figure(figsize = (8,6), constrained_layout=True)
ax = plt.axes(projection='3d')
ax.view_init(elev=20, azim=225)

ax.set_xlabel("Time to maturity $\\tau$")
ax.set(xlim = [0,T])
ax.tick_params(axis='x', pad=0)
ax.xaxis.pane.fill = False

ax.set_ylabel("Stock price $S$")              
ax.set(ylim = [Smin, Smax])
ax.tick_params(axis='y', pad=0)
ax.yaxis.pane.fill = False

ax.zaxis.pane.fill = False

ax.set(title = "Backward Euler approximation for butterfly option")

surf = ax.plot_surface(taus, Ss, V, rstride=1, cstride=1, cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [4]:
fig, ax = plt.subplots(figsize = (12,7))

# approximation
ax.plot(S, V[-1], marker = '.')

# plot cosmetics
ax.set(title = f"Backward Euler approximation at $\\tau={T}$ for butterfly option")
ax.set(xlabel="Stock price $S$", xlim=(Smin, Smax))
ax.xaxis.grid(True)
ax.yaxis.grid(True)

# 2. Bull spread (Dirichlet boundary condition)

The payoff of a *bull spread* with strike $K$ and spread $L$ is as
follows: $$ f(S) =
\begin{cases}
0 & \text{if } S < K, \\
S - K & \text{if } S \in [K, K+L], \\
L & \text{if } S > K+L.
\end{cases}$$

<span class="theorem-title">**Exercise 1**</span> Write a function that
implements the payoff of a bull spread with strike $K$ and spread $L$.
Then create a plot in order to examine the payoff when $K=100$ and
$L=20$.

In [5]:
# Insert code here

The form of the payoff suggests that the value function $V$ of the bull
spread satisfies $$ \lim_{S\rightarrow 0} V(\tau, S) = 0$$ and
$$\lim_{S\rightarrow \infty} V(\tau, S) = e^{-r\tau} L$$ for all
$\tau\in[0,T]$. The appearance of the term $e^{-r\tau}$ is due to the
time value of money (i.e. discounting).

Adapt the code above so that it can be applied to the bull spread.
Recall that the iterative scheme for the Backward Euler method reads $$
    \begin{bmatrix}
d_1 & u_1 & 0 & \cdots & 0 \\
l_2 & d_1 & u_2 & \ddots & \vdots \\
0 & \ddots & \ddots & \ddots & 0 \\
\vdots & \ddots & l_{N-2} & d_{N-2} & u_{N-2} \\
0 & \cdots & 0 & l_{N-1} & d_{N-1}
\end{bmatrix}
\begin{bmatrix}V_{i+1,1}\\ \vdots \\ V_{i+1,N-1} \end{bmatrix}
+ \begin{bmatrix}l_1 V_{i+1,0}\\0\\\vdots\\0\\ u_{N-1}V_{i+1,N} \end{bmatrix} = \begin{bmatrix}V_{i,1}\\ \vdots \\ V_{i,N-1} \end{bmatrix}.
$$

<span class="theorem-title">**Exercise 2**</span> Adapt the code for the
butterfly option to price a bull spread with $K=100$ and $L=20$. Then
display your results graphically.

Hint: Indices in the equation above start at 1 to match the lecture
slides, but the indices of the NumPy arrays containing the diagonals
start at 0.

In [7]:
# Insert code here.

# 3. Call option (Neumann boundary condition)

The payoff of a *call option* with strike $K$ is as follows: $$ f(S) =
\begin{cases}
0 & \text{if } S < K, \\
S - K & \text{if } S \ge K.
\end{cases}$$

<span class="theorem-title">**Exercise 3**</span> Adapt the code given
above to approximate the price of a call option with strike $K=100$. Use
the boundary condition
$$\lim_{S\rightarrow\infty} \frac{\partial^2 V}{\partial S^2}(\tau,S) = 0. $$

In [11]:
#Insert code here